<a href="https://colab.research.google.com/github/denerramosmk/Bootcamp_analistadados_SoulCode/blob/main/Projeto_03_Case_Airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Projeto 03 - Case Airbnb**

**Escola:** SoulCode Academy

**Curso:** Bootcamp Analista de Dados - Martech - AD2

**Professores:** Franciane Rodrigues

**Aluno:** Dener Ramos

## **Objetivo**


Nível - Tratamento e Análise
* Verificar todos os passos da estratégia de análise geral de dados;
* Executar o que for necessário;
* Todos os passos devem ser comentados;
* Realizar no mínimo 3 análises apenas numéricas;
* Realizar plotagens para no mínimo 3 análises;
* Chegar a uma conclusão de sua análise;

Nivel Infra

* O Dataset deve ser salvo em ambiente local  (Google Drive) - (bruto e tratado)
* O Dataset deve ser salvo em ambiente cloud (Cloud Storage) - (bruto e tratado) (será mostrado em aula!)
* O Arquivo original e tratado deve ser salvo em MongoDB Atlas em coleções diferentes (será mostrado em aula!)

## **Base de Dados**

Base de dados com dados de contratações do serviço Airbnb  incluindo preço, tipo de quarto, status do host, e informação de localização.

**Colunas do Dataset:**

[nome original] - [nome traduzido] : [descrição]

* realSum - preço : 	O valor total pago pela reserva em euros

* room_type - tipo_quarto : O tipo de quarto ofertado (pode ser: quarto privado, quarto compartilhado, casa/apt completo)

* room_shared - compartilhado : Informa se o quarto é compartilhado ou não

* person_capacity - capacidade : O máximo de pessoas que podem ser acomodadas em uma única reserva

* host_is_superhost - superhost : Informa se a reserva é de um Superhost da plataforma

* cleanliness_rating - limpeza : Avaliação relacionada a limpeza dada pelos hóspedes

* guest_satisfaction_overall - nota :	Avaliação geral do hóspede sobre sua estadia

* bedrooms - quartos : Número de quartos disponíveis para reserva

* dist - distancia_centro :	Distância em quilômetros da reserva para o centro da cidade

* metro_dist - metro_dist :	Distância em quilômetros da reserva para a estação de metrô mais próxima


Drops:
* room_private : Informa se o quarto é privado ou não
* biz :	Informa se a reserva oferece suporte para negócios como uma sala de conferências.
* multi : Indica disponibilidade de múltiplos quartos dentro de um único anúncio. (um apartamento pode ter dois quartos disponíveis para aluguel)
* lng	: Longitude de cada reserva
* lat	: Latitude de cada reserva
* attr_index_norm
* rest_index
* rest_index_norm



##### Material de consulta:
* https://zenodo.org/record/4446043#.Y9Y9ENJBwUE
* https://www.kaggle.com/datasets/thedevastator/airbnb-price-determinants-in-europe

###### **Pergunta de negócio**

Faça uma análise exploratória de dados com a base de dados do Airbnb e indique possíveis oportunidades e/ou ameaças para o empreendimento.

Análise de base de dados Airbnb, utilizando python e suas bibliotecas para identificar pontos de melhoria ou de atenção que possam ser observados através dos dados.

* medidas descritivas de preço, nota limpeza e nota geral (talvez distancia do centro)
* grafico preço boxsplot




In [ ]:
# Instalação de bibliotecas adicionais
!pip install gcsfs
!pip install pandera

In [ ]:
# Abertura de bibliotecas
import os
import pandas as pd
import numpy as np
import pandera as pa
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from  google.cloud import storage

In [ ]:
# CONFIGURANDO DA CHAVE DE SEGURANCA - ACESSO O PROJETO
serviceAccount = '/content/Chave.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# Configurações Google Cloud Storage - ACESSO AO BUCKET
client = storage.Client()
bucket = client.get_bucket('kraftwerk')
bucket.blob('airbnb.csv')
path = 'gs://kraftwerk/projeto_airbnb/bruto/airbnb.csv'

In [ ]:
# Abertura da base de dados e cópia de segurança (bucket)
df = pd.read_csv(path)
backup1 = df.copy()

In [ ]:
# Comando de abertura para Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# Configuração da quantidade de colunas para aparecer em um DataFrame
pd.set_option('display.max_columns',50) # pd chama a biblioteca pandas, definindo o máximo de colunas a serem exibidas em 50

In [ ]:
# Definindo o dataframe
#df = pd.read_csv('/content/drive/MyDrive/airbnb.csv')

In [ ]:
#backup1 = df.copy() # Criando backup antes de qualquer alteração na tabela


A tabela não possui valores diretamente repetidos.
Aparentemente os valores 'room_shared' e 'room_private' são apenas o oposto um do outro, um indicando se o quarto é o quarto é compartilhado e o outro indicando se é um quarto individual, respectivamente.

Os valores 'attr_index_norm', 'rest_index' e 'rest_index_norm' não possuem documentação no dicionário e não parecem ser necessárias para essa análise.



In [ ]:
ftcompartilhado = df.room_shared == True
ftprivado = df.room_private == True

df.loc[ftcompartilhado & ftprivado]

#Podemos dropar uma das colunas

In [ ]:
# Visualiza valores unicos para ver inconsistências. Utilize a função sorted() para ordenação
print(sorted(pd.unique(df['realSum'])))
print(sorted(pd.unique(df['room_type'])))
print(sorted(pd.unique(df['room_shared'])))
print(sorted(pd.unique(df['room_private'])))
print(sorted(pd.unique(df['person_capacity'])))
print(sorted(pd.unique(df['host_is_superhost'])))
print(sorted(pd.unique(df['multi'])))
print(sorted(pd.unique(df['biz'])))
print(sorted(pd.unique(df['cleanliness_rating'])))
print(sorted(pd.unique(df['guest_satisfaction_overall'])))
print(sorted(pd.unique(df['bedrooms'])))
print(sorted(pd.unique(df['dist'])))
print(sorted(pd.unique(df['metro_dist'])))


In [ ]:
# Drops
df.drop(['attr_index',
         'attr_index_norm',
         'rest_index',
         'rest_index_norm',
         'lng', # Dropando agora, depois de fazer o básico tentar usar
         'lat', # Dropando agora, depois de fazer o básico tentar usar
         'room_private', # Já temos uma coluna que marca se o quarto é compartilhado
         'multi', # Indica se o aluguel é para vários quartos, dropada por não se encaixar na análise que estou fazendo
         'biz'],axis=1, inplace = True) # Indica se o aluguel possui suporte para encontros empresariais, dropada por não se encaixar na análise que estou fazendo

In [ ]:
df.dtypes

In [ ]:
# Traduções
df.rename(columns = {'realSum':"preço",
                     'room_type':'tipo_quarto',
                     'room_shared':'compartilhado',
                     'person_capacity':'capacidade',
                     'host_is_superhost':'superhost',
                     'cleanliness_rating':'limpeza',
                     'guest_satisfaction_overall':'nota',
                     'bedrooms':'quartos',
                     'dist':'distancia_centro',
                     'Unnamed: 0':'id',
                     'metro_dist':'distancia_metro'}, inplace=True)

# a função 'loc' irá localizar elementos e forçar a substituição do valor
df.loc[df.tipo_quarto == 'Private room', ['tipo_quarto']] = 'Quarto privado'
df.loc[df.tipo_quarto == 'Entire home/apt', ['tipo_quarto']] = 'Casa/Apt completo'
df.loc[df.tipo_quarto == 'Shared room', ['tipo_quarto']] = 'Quarto compartilhado'

In [ ]:
df.dtypes


In [ ]:
#Correção dos tipos
df['capacidade'] = df['capacidade'].astype(int)
df['quartos'] = df['quartos'].astype(int)
df['preço'] = df['preço'].astype(float)

In [ ]:
#Arredondando valores
df['preço'] = df['preço'].round(decimals=2)

In [ ]:
# SCHEMA
schema = pa.DataFrameSchema({'preço': pa.Column(pa.Float),
                            'tipo_quarto': pa.Column(pa.String),
                            'compartilhado': pa.Column(pa.Bool),
                            'capacidade': pa.Column(pa.Int),
                            'superhost': pa.Column(pa.Bool),
                            'limpeza': pa.Column(pa.Float),
                            'nota': pa.Column(pa.Float),
                            'distancia_centro': pa.Column(pa.Float),
                            'distancia_metro': pa.Column(pa.Float),
                            })

# Validar o DataFrame
schema.validate(df)

In [ ]:
# Carregamento
backup2 = df.copy()
df.to_csv('gs://kraftwerk/projeto_airbnb/tratado/airbnb_tratado.csv')

In [ ]:
#medidas descritivas preço
descritiva_preco = df.preço.describe()
media_preco = descritiva_preco.loc['mean']  # média de preço
mediana_preco = np.median(df.preço) #mediana de preço
var_preco = df['preço'].var() # variância de preço
desvio_padrao_preco = descritiva_preco.loc['std']  # desvio padrão de preço
q1preco = descritiva_preco.loc['25%'] # Q1 de preço
q3preco = descritiva_preco.loc['75%'] # Q3 de preço
minimo_preco = descritiva_preco.loc['min']  # valor mínimo de preço
maximo_preco = descritiva_preco.loc['max']  # valor máximo de preço
amplitotal_preco = maximo_preco - minimo_preco # amplitude total de preço
ampliq_preco = q3preco - q1preco # amplitude interquartilica de preço
lsuperior_preco = q3preco + (1.5*ampliq_preco) # limite superior de preço
linferior_preco = q1preco - (1.5*ampliq_preco) # limite inferior de preço

print(f'''\t Medidas Descritivas da Coluna Preço \n A média da coluna é {media_preco:.2f} \n A mediana é {mediana_preco:.2f}
\n A variância é {var_preco:.2f} \n O desvio padrão é {desvio_padrao_preco:.2f}
\n O primeiro quartil é {q1preco:.2f} \n O terceiro quartil é {q3preco:.2f}
\n O menor valor é {minimo_preco:.2f} \n O maior valor é {maximo_preco:.2f}
\n A amplitude total é {amplitotal_preco:.2f} \n A amplitude interquartílica é {ampliq_preco:.2f}
\n O limite superior é {lsuperior_preco:.2f} \n O limite inferior {linferior_preco:.2f}''')

In [ ]:
preco_acima = df.loc[df.preço >= 1178.7 ,['preço']]
preco_acima.info()

In [ ]:
#medidas descritivas limpeza
descritiva_limpeza = df.limpeza.describe()
media_limpeza = descritiva_limpeza.loc['mean'] # média de limpeza
mediana_limpeza = np.median(df.limpeza) # mediana de limpeza
var_limpeza = df['limpeza'].var() # variância de limpeza
desvio_padrao_limpeza = descritiva_limpeza.loc['std'] # desvio padrão de limpeza
q1limpeza = descritiva_limpeza.loc['25%'] # Q1 de limpeza
q3limpeza = descritiva_limpeza.loc['75%'] # Q3 de limpeza
minimo_limpeza = descritiva_limpeza.loc['min'] # valor mínimo de limpeza
maximo_limpeza = descritiva_limpeza.loc['max'] # valor máximo de limpeza
amplitotal_limpeza = maximo_limpeza - minimo_limpeza # amplitude total de limpeza
ampliq_limpeza = q3limpeza - q1limpeza # amplitude interquartílica de limpeza
lsuperior_limpeza = q3limpeza + (1.5 * ampliq_limpeza) # limite superior de limpeza
linferior_limpeza = q1limpeza - (1.5 * ampliq_limpeza) # limite inferior de limpeza

print(f'''\t Medidas Descritivas da Coluna Limpeza \n A média da coluna é {media_limpeza:.2f} \n A mediana é {mediana_limpeza:.2f}
\n A variância é {var_limpeza:.2f} \n O desvio padrão é {desvio_padrao_limpeza:.2f}
\n O primeiro quartil é {q1limpeza:.2f} \n O terceiro quartil é {q3limpeza:.2f}
\n O menor valor é {minimo_limpeza:.2f} \n O maior valor é {maximo_limpeza:.2f}
\n A amplitude total é {amplitotal_limpeza:.2f} \n A amplitude interquartílica é {ampliq_limpeza:.2f}
\n O limite superior é {lsuperior_limpeza:.2f} \n O limite inferior {linferior_limpeza:.2f}''')

In [ ]:
limpeza_abaixo = df.loc[df.limpeza < 7.50, ['limpeza']]
limpeza_abaixo.info()

In [ ]:
#medidas descritivas nota
descritiva_nota = df.nota.describe()
media_nota = descritiva_nota.loc['mean'] # média de nota
mediana_nota = np.median(df.nota) # mediana de nota
var_nota = df['nota'].var() # variância de nota
desvio_padrao_nota = descritiva_nota.loc['std'] # desvio padrão de nota
q1nota = descritiva_nota.loc['25%'] # Q1 de nota
q3nota = descritiva_nota.loc['75%'] # Q3 de nota
minimo_nota = descritiva_nota.loc['min'] # valor mínimo de nota
maximo_nota = descritiva_nota.loc['max'] # valor máximo de nota
amplitotal_nota = maximo_nota - minimo_nota # amplitude total de nota
ampliq_nota = q3nota - q1nota # amplitude interquartílica de nota
lsuperior_nota = q3nota + (1.5 * ampliq_nota) # limite superior de nota
linferior_nota = q1nota - (1.5 * ampliq_nota) # limite inferior de nota

print(f'''\t Medidas Descritivas da Coluna Nota \n A média da coluna é {media_nota:.2f} \n A mediana é {mediana_nota:.2f}
\n A variância é {var_nota:.2f} \n O desvio padrão é {desvio_padrao_nota:.2f}
\n O primeiro quartil é {q1nota:.2f} \n O terceiro quartil é {q3nota:.2f}
\n O menor valor é {minimo_nota:.2f} \n O maior valor é {maximo_nota:.2f}
\n A amplitude total é {amplitotal_nota:.2f} \n A amplitude interquartílica é {ampliq_nota:.2f}
\n O limite superior é {lsuperior_nota:.2f} \n O limite inferior {linferior_nota:.2f}''')

In [ ]:
nota_abaixo = df.loc[df.nota < 83, ['nota']]
nota_abaixo.info()

In [ ]:
preco_nota = pd.merge(df.preço, df.nota, on=df.id, how='inner') # Criando uma lista com preço e nota
preco_limpeza = pd.merge(df.preço, df.limpeza, on=df.id, how='inner') # Criando uma lista com preço e limpeza
nota_limpeza = pd.merge(df.nota, df.limpeza, on=df.id, how='inner') # Criando uma lista com preço e limpeza

In [ ]:
ftpreco_nota = preco_nota.loc[(preco_nota.nota < 83) & (preco_nota.preço  >= 1177), ['nota', 'preço']] # 2 id
ftpreco_limpeza = preco_limpeza.loc[(preco_limpeza.limpeza < 7.6) & (preco_limpeza.preço  >= 1177), ['limpeza', 'preço']] #nada
nota_limpeza = nota_limpeza.loc[(nota_limpeza.limpeza < 7.6) & (nota_limpeza.nota  <83), ['limpeza', 'nota']] #nada


In [ ]:
ftpreco_nota

In [ ]:
# Eliminando outliers
df.drop(df[df.preço >= 1178.7].index, axis=0, inplace=True) # Não há valores negativos para tratamento do limite inferior
df.drop(df[df.limpeza < 7.50].index, axis=0, inplace=True) # Não há notas acima de 10 para tratamento do limite superior
df.drop(df[df.nota < 83].index, axis=0, inplace=True) # Não há notas acima de 100 para tratamento do limite superior



In [ ]:
# Gráfico de correlação de todos os elementos numéricos
# Não consegui filtrar só os interessantes
plt.figure(figsize=(14,8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='Blues')

In [ ]:
grafico = {
    'Nota': df.nota.head(100),
    'Limpeza': df.limpeza.head(100),

}
ax = pd.DataFrame(grafico).plot.area(stacked=True)


In [ ]:
#grafico_barra = pd.DataFrame(
#    {'Nota': df['nota'],
#     'Limpeza': df['limpeza']}, index=(df['tipo_quarto']))
#ax = grafico_barra.plot.bar(rot=90)

# Deixei esse gráfico errado aqui pq ainda quero aprender como fazer ele funcionar

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df['distancia_centro'], df['preço'])
plt.xlabel('Distância do Centro')
plt.ylabel('Preço')
plt.title('Relação entre Preço e Distância do Centro')
plt.show()


##### Observações
* É necessário prestar atenção aos outliers. São 56 reservas com preço acima do limite superior; 23 reservas com nota de limpeza abaixo do limite inferior; 48 reservas com nota abaixo do limite inferior. Ainda temos 2 id's que ao mesmo tempo acima do limite superior de preço e abaixo do limite inferior de nota. São 9.71% de outliers se destacando de maneira negativa.


* A nota geral e a limpeza estão muito relacionadas.

* Reservas feitas em quartos individuais, limpos, de um superhost e com mais quartos disponíveis recebem notas melhores.

* A maior oferta de reservas está próxima ao centro, lá tambem estão os valores mais altos

